In [1]:
pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 952.0 kB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import re
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from deep_translator import GoogleTranslator
from google.colab import files
import pickle

In [3]:
from google.colab import files
files.upload()

Saving dataset_new_4aspect.csv to dataset_new_4aspect.csv


{'dataset_new_4aspect.csv': b"content,score,aspek,sentimen\nsangat bagus untuk membantu bapak ibu guru dalam melihat datanya dengan cepat,5,Lainnya,Positif\nsangat membantu app ini karena kami membutuhkan informasi trimakasih simpatika,5,Lainnya,Positif\naplikasi paling top keren banget lengkap yang membuat aplikasi pintar banget sudah diberi kecerdasan ilmu dan mau berbagi terima kasih semoga sukses selalu,5,Lainnya,Positif\nalhamdulillah sangat membantu,5,Lainnya,Positif\nsaya berikan bintang karena semua ikhtiar demi kebaikan walaupun masih perlu pengembangan saya berikan apresiasi yang setinggi-tingginya terima kasih app ini sangat bermanfaat bagi kami para guru di seluruh nusantara,5,Lainnya,Positif\nboleh saya kasih full bintang tapi ada syaratnya hehe tolong permudah saya ketika saya lupa password karena sudah berbagai cara apapun saya ikuti instruksi untuk reset password namun belum berhasil juga hingga sekarang,5,Autentikasi,Negatif\naplikasi simpel dan mudah dipahami mudah-mu

In [11]:
df = pd.read_csv("dataset_new_4aspect.csv")
df

,content,score,aspek,sentimen
0,sangat bagus untuk membantu bapak ibu guru dal...,5,Lainnya,Positif
1,sangat membantu app ini karena kami membutuhka...,5,Lainnya,Positif
2,aplikasi paling top keren banget lengkap yang ...,5,Lainnya,Positif
3,alhamdulillah sangat membantu,5,Lainnya,Positif
4,saya berikan bintang karena semua ikhtiar demi...,5,Lainnya,Positif
...,...,...,...,...
1636,bagus,1,Lainnya,Netral
1637,ok,1,Lainnya,Netral
1638,sukses,1,Lainnya,Netral
1639,sip,1,Lainnya,Netral


In [ ]:
df

,content,score,aspek,sentimen
0,very good to help ladies and gentlemen in seei...,5,Lainnya,Positif
1,really helps this app because we need informat...,5,Lainnya,Positif
2,The most complete top complete application tha...,5,Lainnya,Positif
3,"Alhamdulillah, very helpful",5,Lainnya,Positif
4,I give a star because all the endeavors for go...,5,Lainnya,Positif
...,...,...,...,...
1636,Good,1,Lainnya,Netral
1637,Ok,1,Lainnya,Netral
1638,success,1,Lainnya,Netral
1639,sip,1,Lainnya,Netral


## Translate to English

In [12]:
# Translate to English
def translate_text(text):
    return GoogleTranslator(source='id', target='en').translate(text)

df['content'] = df['content'].apply(translate_text)

## Preprocessing

In [13]:
# Preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df['content'] = df['content'].apply(clean_text)

## Encode labels sentimen

In [14]:
# Encode labels sentimen
sentiment_labels = {"Positif": 2, "Netral": 1, "Negatif": 0}
df['sentimen'] = df['sentimen'].map(sentiment_labels)

## Encode labels aspek

In [8]:
# Encode labels sentimen
aspek_labels = {"Positif": 2, "Netral": 1, "Negatif": 0}
df['aspek'] = df['aspek'].map(aspek_labels)

## Handling Imbalance Data

In [15]:
# Handling Imbalance Data
X = df['content']
y = df['sentimen']

In [16]:
under_sampler = RandomUnderSampler(sampling_strategy={2: 700})
over_sampler = RandomOverSampler(sampling_strategy={0: 700, 1: 700})

X_resampled, y_resampled = under_sampler.fit_resample(X.to_frame(), y)
X_resampled, y_resampled = over_sampler.fit_resample(X_resampled, y_resampled)

## Tokenization

In [17]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_resampled['content'])
vocab_size = len(tokenizer.word_index) + 1

In [18]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(X_resampled['content'])
max_length = max(len(seq) for seq in sequences)
X_padded = pad_sequences(sequences, maxlen=max_length, padding='post')

### Save tokenizer & max_length sentimen

In [23]:
# Save tokenizer
with open('tokenizer_sentimen.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [24]:
# Save max_length
with open('max_length_sentimen.pkl', 'wb') as f:
    pickle.dump(max_length, f)

### Save tokenizer & max_length aspek

In [ ]:
# Save tokenizer
with open('tokenizer_aspek.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
# Save max_length
with open('max_length_aspek.pkl', 'wb') as f:
    pickle.dump(max_length, f)

## Import & Load google word2vec

In [19]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [20]:
# Load Google Word2Vec
word2vec_path = "/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [25]:
# Create embedding matrix
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

In [26]:
# CNN Model
model = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
# Train model
model.fit(X_padded, y_resampled, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.6284 - loss: 0.8288 - val_accuracy: 0.4405 - val_loss: 1.1028
Epoch 2/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.8487 - loss: 0.4565 - val_accuracy: 0.6357 - val_loss: 0.8394
Epoch 3/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.9180 - loss: 0.2654 - val_accuracy: 0.7429 - val_loss: 0.5554
Epoch 4/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9533 - loss: 0.1737 - val_accuracy: 0.7738 - val_loss: 0.4695
Epoch 5/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - accuracy: 0.9487 - loss: 0.1527 - val_accuracy: 0.7857 - val_loss: 0.4579
Epoch 6/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.9563 - loss: 0.1272 - val_accuracy: 0.8095 - val_loss: 0.2984
Epoch 7/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.9495 - loss: 0.1300 - val_accuracy: 0.7857 - val_loss: 0.3463
Epoch 8/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.9573 - loss: 0.1093 - val_accuracy: 0.7881 - v

In [28]:
# Save model
model.save('cnn_sentiment_model.h5')

In [31]:
def predict_sentiment(text):
    text = translate_text(text)
    text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    prediction = model.predict(padded_sequence)
    return {0: 'Negatif', 1: 'Netral', 2: 'Positif'}[np.argmax(prediction)]

In [39]:
# Contoh prediksi
print(predict_sentiment(""))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Negatif
